In [ ]:
# 【3780】hog svm line训练【3780】
#。hog*svm = 值
# 值》T 目标obj 

In [ ]:
#整体hog cell复用
# 3780
# 3780 《-win（block cell bin）
# 1《-bin
# cell0 cell3 bin0-bin8
# cell0: bin0 bin1 。。。bin8
# cell1: bin0 bin1 。。。bin8
# cell2: bin0 bin1 。。。bin8
# cell3: bin0 bin1 。。。bin8
#ij cell0 bin0=《f0，
#i+1 j cell0 bin0 = f1
#ij。。。。
# sumbin0（f0+f1.。）= bin0
# 权重累加
#ij bin0 bin1 

# cell复用

# block 4个cell
# 【0】【】【】【3】
# cell0 bin0-bin9 
# cellx0 cellx2 cellx4
# cellx0:ij-》bin bin+1
# cellx2：ij -》 cell2 cell3 -》bin bin+1 bin bin+1
# cellx4：ij

# 【cell 9】【4cell】【105】 = 3780

In [ ]:
# bin 投影 梯度
# bin 0-360 9bin 0-40
# bin1 0-20 180-200
# ij f a = 10
# 0-20 center bin1 a=190 180 200 bin1
# f 
# 25 bin1 bin2 
# f1 = f*f（夹角） f2 = f*（1-f（夹角））  f（夹角）  0-1.0
# +1 hog 

## hog计算

车辆检测，可以根据windows窗体计算，判决这是车辆还是不是车辆。

可以计算出一个windows窗体下有多少个block。

block 下 cell 是不可滑动的。

### cell bin 梯度 运算

每个像素都有一个梯度: 大小(浮值) 方向

当前的bin 与方向有关

0-360度，以40度为一个bin进行划分，可以分出九个bin。

一个bin = 40度

cell和bin的关系。一个cell必须完整的包含360度的信息，多以就必须让cell包含九个bin

一个cell中对应9个bin

![mark](http://myphoto.mtianyan.cn/blog/180327/je88mgjB7l.png?imageslim)

### hog特征的维度

haar特征是一个具体的值，白-黑

我们的hog特征是一个向量，就会有一个维度的问题: 必须完全描述一个obj的所有信息

维度 = 每个windows窗体中有多少个block,cell个数 (105,4,9)=3780

### 梯度方向大小

必须以像素为单位，每个像素都有一个梯度，所有的像素共同构成hog特征

windows窗体下的所有像素。

运算量很大，hog的特征模板 -> haar类似

它的模板分两种: 水平和竖直

```
[1,0,-1] [[1],[0],[-1]]
水平方向上，左中右三个数分别与模板相乘
a = p1*1+p2*0+p3*(-1) = 相邻像素之差
b = 上下像素之差
f = 根号下（a方+b方）
angle = arctan（a/b）
```

In [ ]:
#7 cell bin 梯度：运算
# 每个像素-》梯度 ：大小 f 方向 angle
# 0-360 /40 = 9块 = 9bin
# 1bin = 40 cell-》360-〉9bin
# hog特征维度：
# haar 值 hog 向量 （维度）-》完全描述 一个obj info all
# 维度 = 105*4*9=3780 

## hog特征

什么是hog特征

- 什么是hog？

>特征: 某个特定区域的像素进行某种特定运算得到的结果

haar特征是直接经过模板计算的结果: 我们称之为haar特征。

步骤:  

1. 模块划分 
2. 计算梯度和方向 模版 
3. 使用梯度和方向进行bin投影 
4. 计算每个模块的hog特征

### 模块划分

![mark](http://myphoto.mtianyan.cn/blog/180327/2gCimdc32d.png?imageslim)

整个蓝色方框是我们图片的大小，我们还有一个windows窗口。窗口我们也设置为当前大小。

![mark](http://myphoto.mtianyan.cn/blog/180327/IBcKHFGJLc.png?imageslim)

- image（ppt蓝色区域） windows窗体（蓝色） block模块（红色） cell （绿色）

每个概念都有一个size

关系: image > win > block >cell

滑动窗口概念

block-step  win-step 

>cell-bin

一般情况下: 窗体是特征计算的最顶层单元(一个窗口正常情况要能覆盖车的全部信息) 
必须包含一个目标obj的所有描述信息

```
1. win-size 50*100 20*50 推荐大小64*128
2. block(红)  小于win大小  win-size 宽高是block整数倍
当窗体大小为64*128时，通常的block为16*16
3. block-step block会在窗体中从上到下，从左到右进行遍历。
block每次滑动多少个像素。8*8

计算block-cout(滑动多少个block)= （（64-16）/8+1）*（（128-16）/8+1）= 
105 block

水平竖直两个方向滑动步长。

5. cell size 8*8 
6. block = ?cell 16*16 = 2*2 = > 4cell  cell1-cell4 不可滑动
7. bin?
```